#### Пример 17.1. sentence.py: объект Sentence как последовательность слов

In [2]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self):
        return len(self.words)

    def __repr__(self):
        return f'Sentence{reprlib.repr(self.text)}'
        

#### Пример 17.2. Итерирование объекта Sentence

In [3]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence'"The time ha... Walrus said,'

In [4]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [5]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [6]:
s[0]

'The'

In [7]:
s[5]

'Walrus'

In [8]:
s[-1]

'said'

#### Почему последовательности итерируемы: функция iter

In [12]:
class Spam:
    def __getitem__(self, i):
        print('->', i)
        raise IndexError()

spam_can = Spam()

In [13]:
iter(spam_can)

In [14]:
list(spam_can)

-> 0


[]

In [12]:
from collections import abc
isinstance(spam_can, abc.Iterable)

False

In [1]:
class GooseSpam:
    def __iter__(self):
        pass

from collections import abc
issubclass(GooseSpam, abc.Iterable)

True

In [2]:
goose_spam_can = GooseSpam()
issubclass(GooseSpam, abc.Iterable)

True

#### Использование iter в сочетании с Callable

In [16]:
from random import randint
def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)
d6_iter

In [17]:
for roll in d6_iter:
    print(roll)

3
6
3
6
3
4
4
2
4
4
2
2


#### Итерируемые объекты и итераторы

In [18]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [19]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


#### Пример 17.3. Класс abc.Iterator; код взят из файла Lib/_collections_abc.py (https://github.com/python/cpython/blob/b1930bf75f276cd7ca08c4455298128d89adf7d1/Lib/_collections_abc.py#L271)

In [20]:
class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):
        return self

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            return _check_methods(C, '__iter__', '__next__')
        return NotImplemented

NameError: name 'Iterable' is not defined

In [21]:
s3 = Sentence('Life of Brian')
it = iter(s3)
it

In [21]:
next(it)

'Life'

In [22]:
next(it)

'of'

In [23]:
next(it)

'Brian'

In [24]:
next(it)

StopIteration: 

In [25]:
list(it)

[]

In [26]:
list(iter(s3))

['Life', 'of', 'Brian']

#### Пример  17.4. sentence_iter.py: класс Sentence, реализованный с помощью паттерна Итератор

In [30]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence{reprlib.repr(self.text)}'

    def __iter__(self):
        return SentenceIterator(self.words)

class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0

    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

    def __iter__(self):
        return self

#### Пример 17.5. sentence_gen.py: реализация класса Sentence с помощью генератора

In [31]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence{reprlib.repr(self.text)}'

    def __iter__(self):
        for word in self.words:
            yield word

#### Пример 17.6. Генераторная функция, отдающая три числа

In [22]:
def gen_123():
    yield 1
    yield 2
    yield 3

gen_123 # doctest: +ELLIPSIS

<function __main__.gen_123()>

In [23]:
gen_123() # doctest: +ELLIPSIS

<generator object gen_123 at 0x7f4269cf8cf0>

In [24]:
for i in gen_123():
    print(i)

1
2
3


In [25]:
g = gen_123()
next(g)

1

In [26]:
next(g)

2

In [27]:
next(g)

3

In [28]:
next(g)

StopIteration: 

#### Пример 17.7. Генераторная функция, печатающая сообщения во время выполнения

In [29]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end')

for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end


In [2]:
def gen_AB():
    print('start')
    print('continue')
    print('end')

for c in gen_AB():
    print('-->', c)

start
continue
end


TypeError: 'NoneType' object is not iterable

#### Пример  17.8. sentence_gen2.py: реализация класса Sentence с помощью генераторной mфункции, которая вызывает генераторную функцию re.finditer

In [1]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  # <1>

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):  # <2>
            yield match.group()  # <3>

#### Пример 17.9. Генераторная функция gen_AB используется сначала в списковом включении, а затем в генераторном выражении

In [2]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

In [3]:
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [7]:
res1

['AAA', 'BBB']

In [4]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [5]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x7fb6d04d7e60>

In [6]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


#### Пример 17.10. sentence_genexp.py: реализация класса Sentence с помощью генераторного выражения

In [9]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

#### Пример 17.12. Класс ArithmeticProgression